In [ ]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import awkward as ak
import matplotlib.pyplot as plt
import numba
from numba import njit
import numpy as np

In [ ]:
filename ="/pnfs/psi.ch/cms/trivcat/store/mc/Run3Summer22EENanoAODv12/VBFHHto4B_CV_1_C2V_1_C3_1_TuneCP5_13p6TeV_madgraph-pythia8/NANOAODSIM/130X_mcRun3_2022_realistic_postEE_v6-v3/80000/a897cbbf-ea0b-40d5-b850-f9f24a7906a5.root"
# events = NanoEventsFactory.from_root(filename, schemaclass=NanoAODSchema).events()
events = NanoEventsFactory.from_root(filename, schemaclass=NanoAODSchema, entry_stop=20000).events()
print("Events read:", len(events))

In [ ]:
events.GenPart=ak.with_field(events.GenPart, ak.local_index(events.GenPart, axis=1), "index")
genpart= events.GenPart
single_event=-1
if (single_event!=-1):
    genpart= events.GenPart[single_event]


isQuark = abs(genpart.pdgId) < 7
isHard = genpart.hasFlags(["fromHardProcess"])

quarks = genpart[isQuark & isHard]
quarks = quarks[quarks.genPartIdxMother!=-1]

quarks_mother = genpart[quarks.genPartIdxMother]
quarks_mother_children = quarks_mother.children
quarks_mother_children_isH = ak.sum((quarks_mother_children.pdgId == 25), axis=-1)==2
vbf_quarks = quarks[quarks_mother_children_isH]


print(quarks_mother_children_isH, quarks_mother_children_isH[0])
print(quarks.pdgId, quarks.genPartIdxMother, quarks.index)
print(vbf_quarks.pdgId, vbf_quarks.genPartIdxMother, vbf_quarks.index, genpart[vbf_quarks.genPartIdxMother].pdgId)
print(vbf_quarks.hasFlags(["isFirstCopy"]))


In [ ]:

children_idxG = ak.without_parameters(genpart.childrenIdxG, behavior={})
children_idxG_flat = ak.flatten(children_idxG, axis=1)
genpart_pdgId_flat = ak.flatten(ak.without_parameters(genpart.pdgId, behavior={}), axis=1)
genpart_LastCopy_flat = ak.flatten(ak.without_parameters(genpart.hasFlags(["isLastCopy"]), behavior={}), axis=1)
genpart_pt_flat = ak.flatten(ak.without_parameters(genpart.pt, behavior={}), axis=1)
genparts_flat = ak.flatten(genpart)
genpart_offsets = np.concatenate([[0],np.cumsum(ak.to_numpy(ak.num(genpart, axis=1), allow_missing=True))])
local_index_all = ak.local_index(genpart, axis=1)
local_index_vbf = ak.local_index(vbf_quarks, axis=1)
vbf_quark_idx = ak.to_numpy(vbf_quarks.index+genpart_offsets[:-1], allow_missing=False)
vbf_quarks_pdgId = ak.to_numpy(vbf_quarks.pdgId, allow_missing=False)
nevents=vbf_quark_idx.shape[0]


In [ ]:
print(genpart_offsets)
print(children_idxG)
print(children_idxG_flat)
print(genpart_pdgId_flat)
print(genpart_LastCopy_flat)
print(genparts_flat.index, ak.num(genparts_flat.index, axis=0))
print(local_index_all, ak.num(local_index_all, axis=1))
print(nevents)
print(vbf_quark_idx)

In [ ]:
@njit
def analyze_parton_from_vbf_quarks(
    vbf_quarks_idx,
    vbf_quarks_pdgId,
    children_idxG_flat,
    genpart_pdgId_flat,
    genpart_offsets,
    genpart_LastCopy_flat,
    genpart_pt_flat,
    nevents,
):
    prints=False
    # print input array
    if prints:
        print("vbf_quarks_idx", vbf_quarks_idx)
        print("vbf_quarks_pdgId", vbf_quarks_pdgId)
        print("children_idxG_flat", children_idxG_flat)
        print("genpart_pdgId_flat", genpart_pdgId_flat)
        print("genpart_offsets", genpart_offsets)
        print("genpart_LastCopy_flat", genpart_LastCopy_flat)
        print("nevents", nevents)

    # get the children ofthe vbf_quarks which have the same pdgId of the mother iteratively until we reach the last copy

    out = np.zeros(vbf_quarks_idx.shape, dtype="int64")-1

    for iev in range(vbf_quarks_idx.shape[0]):
        if prints: print("Event", iev)
        for ipart in range(vbf_quarks_idx.shape[1]):
            p_id = vbf_quarks_idx[iev][ipart]
            if prints: print("Parton", ipart)
            i=0
            while not genpart_LastCopy_flat[p_id] and i<5:
                i+=1
                children_idxs = children_idxG_flat[p_id]
                if prints: print(children_idxs)

                #get the children with the same pdgId as the mother with highest pt
                max_pt = -1
                max_pt_idx = -1
                if genpart_LastCopy_flat[p_id]:
                    out[iev][ipart] = p_id
                    continue


                # num_pdg_equal=0
                # for child_idx in children_idxs:
                #     if genpart_pdgId_flat[child_idx] == vbf_quarks_pdgId[iev][ipart]:
                #         num_pdg_equal+=1
                # if num_pdg_equal==1:
                #     p_id = children_idxs[0]
                #     break
                # if prints: print("\n\n###################\n\n")


                for child_idx in children_idxs:
                    if prints: print("Child", child_idx)
                    if prints: print(vbf_quarks_pdgId[iev][ipart])
                    if prints: print(genpart_pdgId_flat[child_idx])
                    if genpart_pdgId_flat[child_idx] != vbf_quarks_pdgId[iev][ipart]:
                        continue
                    child_pt = genpart_pt_flat[child_idx]
                    if prints: print(child_pt)
                    if child_pt > max_pt:
                        max_pt_idx = child_idx
                        max_pt = child_pt

                if prints: print("genpart_LastCopy_flat", genpart_LastCopy_flat[max_pt_idx])
                if  genpart_LastCopy_flat[max_pt_idx]:
                    if prints: print("Found child")
                    out[iev][ipart] = max_pt_idx
                if prints: print(p_id, max_pt_idx)
                p_id = max_pt_idx
                if prints: print(p_id, max_pt_idx)
                # break
                # if max_pt == -1:
                #     max_pt_idx = p_id
                if out[iev][ipart]!=-1:
                    if prints: print("out", out[iev][ipart])
                    break

    return out

In [ ]:
vbf_quark_last_idx=analyze_parton_from_vbf_quarks(
    vbf_quark_idx,
    vbf_quarks_pdgId,
    children_idxG_flat,
    genpart_pdgId_flat,
    genpart_offsets,
    genpart_LastCopy_flat,
    genpart_pt_flat,
    nevents,
)
print(vbf_quark_last_idx)
print(genpart_offsets[:-1])

# Do some tests

In [ ]:
vbf_quark_last_pt=ak.flatten(genparts_flat[vbf_quark_last_idx].pt)
vbf_quark_first_pt=ak.flatten(genparts_flat[vbf_quark_idx].pt)
print(vbf_quark_last_pt)
#plot ratio of pt of last copy of quark to the pt of the quark
plt.hist(vbf_quark_last_pt/vbf_quark_first_pt, bins=100, range=(0,2))

In [ ]:
# plot ratio of eta of last copy of quark to the eta of the quark
vbf_quark_last_eta=ak.flatten(genparts_flat[vbf_quark_last_idx].eta)
vbf_quark_first_eta=ak.flatten(genparts_flat[vbf_quark_idx].eta)
plt.hist(vbf_quark_last_eta/vbf_quark_first_eta, bins=100, range=(0.5,1.5))


In [ ]:
#ratio of pdgId of last copy of quark to the pdgId of the quark
vbf_quark_last_pdgId=ak.flatten(genparts_flat[vbf_quark_last_idx].pdgId)
vbf_quark_first_pdgId=ak.flatten(genparts_flat[vbf_quark_idx].pdgId)
plt.hist(vbf_quark_last_pdgId/vbf_quark_first_pdgId, bins=100, range=(0,2))


# TEST

In [ ]:
quark_last=(genpart[isQuark & isHard & (genpart.genPartIdxMother!=-1)& genpart.hasFlags(["isLastCopy"])])
quark_last_flatten=ak.flatten(quark_last)
print(quark_last.pdgId, quark_last.genPartIdxMother, quark_last.index,len(quark_last_flatten))

In [ ]:
quark_last_mother = (genpart[quark_last.genPartIdxMother])
quark_last_mother_flatten=ak.flatten(quark_last_mother)
print(quark_last_mother.pdgId, quark_last_mother.children.pdgId, quark_last_mother.children.index, len(quark_last_mother_flatten.pdgId))

In [ ]:
# get mother of mother
quark_mother_not_null=quark_last[quark_last_mother.genPartIdxMother!=-1]
quark_mother_not_null_flatten=ak.flatten(quark_mother_not_null)
mother_not_null=quark_last_mother[quark_last_mother.genPartIdxMother!=-1]
mother_not_null_flatten=ak.flatten(mother_not_null)
quark_last_mother_mother = genpart[mother_not_null.genPartIdxMother]
quark_last_mother_mother_flatten=ak.flatten(quark_last_mother_mother)
print(quark_last_mother_mother.pdgId, quark_last_mother_mother.children.pdgId, quark_last_mother_mother.children.index, len(quark_last_mother_mother.pdgId))

In [ ]:
#plot pt ratio of last copy of quark to the pt of the mother
plt.hist(quark_last_flatten.pt/quark_last_mother_flatten.pt, bins=100, range=(0,2))

In [ ]:
plt.hist(mother_not_null_flatten.pt/quark_last_mother_mother_flatten.pt, bins=100, range=(0,2))

In [ ]:
plt.hist(quark_mother_not_null_flatten.pt/quark_last_mother_mother_flatten.pt, bins=100, range=(0,2))

# check if it can happen that multiple children of a particle have the same pdgId has the parent

In [ ]:
isQuark = abs(genpart.pdgId) < 7
isHard = genpart.hasFlags(["fromHardProcess"])

quarks = genpart[isQuark & isHard]

quarks = quarks[quarks.genPartIdxMother!=-1]

quarks_mother = genpart[quarks.genPartIdxMother]
quarks_mother_children = quarks_mother.children
quarks_mother_children_isH = ak.sum((quarks_mother_children.pdgId == 25), axis=-1)==2
vbf_quarks = quarks[quarks_mother_children_isH]

print("vbf_quarks children", vbf_quarks.children.pdgId)


children_pdgId = genpart.children.pdgId
parent_pdgId = genpart.pdgId

print(parent_pdgId)
print(children_pdgId)

for event in range(len(vbf_quarks)):
    # for i in range(len(vbf_quarks[event])):
    for i in vbf_quarks[event].index:
        if (
            # True
            ak.sum(parent_pdgId[event][i] == children_pdgId[event][i]) > 1
            and not ak.any(children_pdgId[event][i] == 25)
            and ak.sum(genpart[event][i].children.hasFlags(["fromHardProcess"])) > 1
        ):
            print("Event", event, "Particle", i)
            print("Parent pdg", parent_pdgId[event][i])
            print("Children pdg", children_pdgId[event][i])
            print("chilren index", genpart[event][i].children.index)
            print("children pt", genpart[event][i].children.pt)
            # print("children is last copy", genpart[event][i].children.hasFlags(["isLastCopy"]))
            print(
                "children from hard",
                genpart[event][i].children.hasFlags(["fromHardProcess"]),
            )
            print(
                "children is hard",
                genpart[event][i].children.hasFlags(["isHardProcess"]),
            )
            print(
                "children is prompt", genpart[event][i].children.hasFlags(["isPrompt"])
            )

            print("parent index", genpart[event][i].index)
            print("childre mother index", genpart[event][i].children.genPartIdxMother)